#### Idea 
* Generate
* Feedback 
* Re-generate

In [1]:
from src.idea import Idea, IdeaPrompt
from src.model import ClaudeModel, OpenAIModel

context = IdeaPrompt.format(context="I am sick, what medicine do I take?", feedback="Be funny")
idea = Idea.make(context=context)


In [2]:
# Present Options
print("Available options:")
for i, option in enumerate(idea.options):
    print(f"{i + 1}. {option}")

# Like & Dislike 
idea.like(0)
print("Liked option 1")

# Edit
new_option = "Try a dose of humor, but consult your doctor for actual medical advice!"
idea.edit(1, new_option)
print(f"Edited option 2 to: {new_option}")

# Add Feedback
feedback = "Laughter doesn't cure diabetes the last time I check"
idea.add_feedback(feedback)
print(f"Added feedback: {feedback}")

# Display updated options and preferences
print("\nUpdated options:")
for i, option in enumerate(idea.options):
    status = "Liked" if idea.preferences[i] else "Not liked"
    print(f"{i + 1}. {option} - {status}")

print(f"\nCurrent feedback: {idea.feedback}")


Available options:
1. Take a spoonful of laughter, it's the best medicine!
2. You need a dose of dad jokes, guaranteed to make you groan and feel better.
3. Try a pun-atic medication, it'll have you rolling on the floor with laughter.
4. A funny movie marathon is the perfect prescription for your sickness.
5. Hire a clown to do a stand-up routine in your living room. Laughter is contagious!
Liked option 1
Edited option 2 to: Try a dose of humor, but consult your doctor for actual medical advice!
Added feedback: Laughter doesn't cure diabetes the last time I check

Updated options:
1. Take a spoonful of laughter, it's the best medicine! - Liked
2. Try a dose of humor, but consult your doctor for actual medical advice! - Not liked
3. Try a pun-atic medication, it'll have you rolling on the floor with laughter. - Not liked
4. A funny movie marathon is the perfect prescription for your sickness. - Not liked
5. Hire a clown to do a stand-up routine in your living room. Laughter is contagiou

In [3]:
idea.regenerate()

In [4]:
# Present Options
print("Available options:")
for i, option in enumerate(idea.options):
    print(f"{i + 1}. {option}")

Available options:
1. Drink a bottle of soda pop - the fizz will cure what fizz you!
2. Take two aspirins and call me in the morning... if you can still dial a phone.
3. Try a handful of jellybeans. The sugar rush might just give you a second wind!
4. Have you tried turning yourself off and back on again? Rebooting might fix the bug.
5. I prescribe a heavy dose of dad jokes. Laughter really is the best medicine, even for diabetes!


#### Use Case
1. Sampling queries to test rule-ahdering ability 

In [ ]:
# test with Chainlit interface (for the manual annotation here)

# For the feedback provision can I just use terminal anyways?
# - i like terminal more than anything else ... 


#### CrewAI 
1. How are you going to take the feedback into account here? Another wrapper layer above it?

Step 1 - Review my post history and recommend improvement opportunities 
- Review my popular posts this year (see below)
- Go article by article and summarise factors in each article which might increase or decrease engagement
- Based on a summary of the factors for each arctle, identify five measures I can take to improve engagement on my next post

[Add context: my selected measures to improve engagement in next post]

Step 2 - Find a topic
- Research AI, data and tech news over the last 15 days and suggest five topics that might appeal to my audience
- Research AI papers publised to Arxiv over the last 15 days and suggest five additional topics that might appeal to my audience 

[Keep repeating with guidance until happy]
[Add context: my selected release/news/paper/topic]

Step 3 - Research
- Do more detailed research now on selected topic
- Suggest ten interesting perspectives that could form the basis for the article
- Find ten key quotes from industry leaders I could use to illustrate the article
- Find twenty facts and figures that can used to support the article

[Keep repeating with guidance until happy]
[Add context: my selected perspectives, quotes & facts + any I chose to add]

Step 4 – Create an Engaging Title
- Compare previous articles and the level of engagement which each engendered to tune the suggestions
- Suggest ten catchy titles for the post which would maximise engagement based on what we have learned from previous articles

[Keep repeating with guidance until happy]
[Add context: my selected title]

Step 5 - Create an Engaging Image
- Compare previous articles and the level of engagement which each engendered to tune the suggestions
- Suggest ten descriptions for images that would maximise engagement based on what we have learned from previous images

[Keep repeating with guidance until happy]
[Add context: my selected image description]

Step 6 – Create a draft of the article
- Create a draft of the LinkedIn article
- Restate the selected title
- Restate the select image description
- Utilise the selected measures to improve engagement
- Start with a hook to grab the reader's attention
- Organize points in a logical sequence
- Expand on each main point in paragraphs
- Include evidence, examples, and explanations
- Ensure each paragraph flows logically to the next
- Manage the length to a good size for LinkedIn
- End with a call to action or a thought-provoking question.

In [3]:
from crewai_tools import SeleniumScrapingTool
from crewai import Agent, Task, Crew, Process
import os 

os.environ["SERPER_API_KEY"] = "5affe20ad27423ecb9954ffa3151680c42adadca"
os.environ["SERPLY_API_KEY"] = "Xr5jNnnGynMUgAWqEC1vtyCB"

scrape_tool = SeleniumScrapingTool()

useful_links = [
  "https://huggingface.co/papers",
  "https://arxiv.org/list/cs/new"
]

researcher = Agent(
  role="Researcher",
  goal=f"""Research the selected paper and suggest five interesting perspectives that could form the basis for the article. Look through provided links to find interesting papers and provide a summary of the main points and the paper.
  Useful links: {useful_links[1]}""",
  backstory="""You are a Research Scientist at Google DeepMind focusing on AI.
  You are good at using the provided link to scrape the paper and summarise the main points, as well as search the web for related introduction to get a better perspectives on how to understand the topic.""",
  verbose=True,
  allow_delegation=True,
  tools=[scrape_tool]
)

task1 = Task(
  description="""Look through the new AI papers inside the provided link and suggest 5 most interesting papers with their link andabstracts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

# Instantiate your crew with a sequential process
crew = Crew(
  # agents=[finder, researcher, writer],
  agents = [researcher],
  # tasks=[task1, task2, task3],
  tasks = [task1],
  verbose=True,
  process = Process.sequential
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

2024-08-09 17:45:45,967 - 7948459712 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-09 17:45:45][DEBUG]: == Working Agent: Researcher
 [2024-08-09 17:45:45][INFO]: == Starting Task: Look through the new AI papers inside the provided link and suggest 5 most interesting papers with their link andabstracts.


> Entering new CrewAgentExecutor chain...
To complete the task of identifying and summarizing five interesting AI papers from the provided link (https://arxiv.org/list/cs/new), I will first read the contents of the website to gather information about the new AI papers.

Action: Read a website content
Action Input: {"website_url": "https://arxiv.org/list/cs/new", "css_element": ".list-identifier a"} 



Thought: To gather the necessary information, I need to read the content of the provided link again and extract the details about the new AI papers.

Action: Read a website content
Action Input: {"website_url": "https://arxiv.org/list/cs/new", "css_element": ".list-title"}
 

Building Machines that Learn and Think with People
Taxonomy Driven Fast Adversarial

In [2]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, JSONSearchTool, SerplyScholarSearchTool, SeleniumScrapingTool, SerplyWebSearchTool

os.environ["SERPER_API_KEY"] = "5affe20ad27423ecb9954ffa3151680c42adadca"
os.environ["SERPLY_API_KEY"] = "Xr5jNnnGynMUgAWqEC1vtyCB"
search_tool = SerperDevTool()
search_post_tool = JSONSearchTool()
scholar_search_tool = SerplyScholarSearchTool()
scrape_tool = SeleniumScrapingTool()
web_search_tool = SerplyWebSearchTool()



analyst = Agent(
  role = "LinkedIn Post Analyst",
  goal = "Go article by article and summarize factors in each article which might increase or decrease engagement. Based on a summary of the factors for each article, identify five measures I can take to improve engagement on my next post.",
  backstory = """You are an expert at analysing LinkedIn posts and providing insights on how to improve them.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose = True,
  allow_delegation = False,
  tools = [search_post_tool]
)



finder = Agent(
  role='Research Analyst',
  goal='Uncover AI Research over the past 15 days and suggests 5 most interesting papers with their links and abstracts.',
  backstory="""You are a Research Analyst at Google DeepMind focusing on AI.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[scholar_search_tool]
)

useful_links = [
  "https://huggingface.co/papers",
  "https://arxiv.org/list/cs/new"
]

researcher = Agent(
  role="Researcher",
  goal=f"""Research the selected paper and suggest five interesting perspectives that could form the basis for the article. Look through provided links to find interesting papers and provide a summary of the main points and the paper.
  Useful links: {useful_links[1]}""",
  backstory="""You are a Research Scientist at Google DeepMind focusing on AI.
  You are good at using the provided link to scrape the paper and summarise the main points, as well as search the web for related introduction to get a better perspectives on how to understand the topic.""",
  verbose=True,
  allow_delegation=True,
  tools=[scrape_tool]
)

writer = Agent(
  role='Key Opinion Leader',
  goal='Write a short post on the selected topic, highlighting the key insights from the research',
  backstory="""You are a Key Opinion Leader at LinkedIn, you are good at writing engaging posts on the selected topic, highlighting the key insights from the research""",
  verbose=True,
  allow_delegation=True
)

# Create tasks for your agents
task1 = Task(
  description="""Look through the new AI papers inside the provided link and suggest 5 most interesting papers with their link andabstracts.""",
  expected_output="Full analysis report in bullet points",
  agent=finder
)

task2 = Task(
  description="""Research the selected paper and suggest ten interesting perspectives that could form the basis for the article.""",
  expected_output="Ten interesting perspectives for the article",
  agent=researcher
)

task3 = Task(
  description="""Using the 5 papers, research and draft 5 short LinkedIn posts to convey highly compressed and intuitive insights to engage better with the audience.""",
  expected_output="5 distinct LinkedIn posts",
  agent=writer
)

# Instantiate your crew with a sequential process
crew = Crew(
  # agents=[finder, researcher, writer],
  agents = [researcher],
  # tasks=[task1, task2, task3],
  tasks = [task1],
  verbose=True,
  process = Process.sequential
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

2024-08-09 17:44:18,661 - 7948459712 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-09 17:44:18][DEBUG]: == Working Agent: Research Analyst
 [2024-08-09 17:44:18][INFO]: == Starting Task: Look through the new AI papers inside the provided link and suggest 5 most interesting papers with their link andabstracts.


> Entering new CrewAgentExecutor chain...
I need to identify the most interesting AI research papers published over the past 15 days. To do this, I'll use the Scholar Search tool to find relevant papers.

Action: Scholar Search
Action Input: {"search_query": "AI research papers published in the past 15 days"} 


Search results: Title: The effects of GnRH treatment at the time of AI and 12 days later on reproductive performance of high producing dairy cows during the warm season in northeastern …
Link: https://www.sciencedirect.com/science/article/pii/S0093691X05002906
Description: F López-Gatius, P Santolaria, A Martino, F Delétang… - Theriogenology, 2006 - Elsevier
Cite: https://scholar.google.com/scholar?q=info:LquYsEOfRDoJ:scholar.google.com/&outp

KeyboardInterrupt: 

In [ ]:
# Can we successfully scrape one web-page and get the information therein? 
from creai_tools import SeleniumScrapingTool, SerperDevTool

scrape_tool = SeleniumScrapingTool()

extractor = Agent(
    role='Web Scraper',
    goal='Scrape the web page and extract the research paper and their abstract',
    backstory="""You are a web scraper who is good at extracting the information from the web page.""",
    verbose=True,
    allow_delegation=False,
    tools=[scrape_tool]
)

task = Task(
    description="""Scrape the web page and extract the research paper and their abstract.
    huggingface.co/papers
    arxiv.org/list/cs/new""",
    expected_output="The research paper and their abstract",
    agent=extractor
)

In [2]:
from src.utils import crawl_arxiv_papers 
from src.utils import get_last_dates 

crawl_arxiv_papers() # This will always crawl the latest 200 papers regarding AI 

# Do a Best-of-N paper selection 

# Process on each paper, come up with a 'talk-to-me-as-I-am-a-five-year-old' like explanation of it, and why it's important

In [1]:
from src.utils import read_quick_info

read_quick_info()

FileNotFoundError: [Errno 2] No such file or directory: 'cave/arxiv_papers_info.json'

In [3]:

import glob 
from src.utils import get_pdf_contents, get_oai_response

# Detailed Read 

# Loading Paper from the Cave 
dir_path = "cave/paper"
files = glob.glob(dir_path + "/*.pdf")

# Reading Paper one by one 
for file in files: 
    imgs = get_pdf_contents(file, first_page=1, last_page=3)
    get_oai_response(prompt="What is the main idea of this paper?", img=imgs, img_type="image/png")
    


In [4]:
import os
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
from src.utils import get_last_dates


os.environ["SERPER_API_KEY"] = "5affe20ad27423ecb9954ffa3151680c42adadca"
os.environ["SERPLY_API_KEY"] = "Xr5jNnnGynMUgAWqEC1vtyCB"

# Loading Tools
search_tool = SerperDevTool()

# Define your agents with roles, goals, tools, and additional attributes
researcher = Agent(
    role='Senior Research Analyst',
    goal='Uncover cutting-edge developments in AI and data science',
    backstory=(
        "You are a Senior Research Analyst at a leading tech think tank. "
        "Your expertise lies in identifying emerging trends and technologies in AI and data science. "
        "You have a knack for dissecting complex data and presenting actionable insights."
        "You usually goto huggingface.co/papers and arxiv.org/list/cs/new to research the latest advancements in AI."
        f"Current date: {current_date}"
    ),
    verbose=True,
    allow_delegation=False,
    tools=[search_tool]
)
writer = Agent(
    role='Tech Content Strategist',
    goal='Craft compelling content on tech advancements',
    backstory=(
        "You are a renowned Tech Content Strategist, known for your insightful and engaging articles on technology and innovation. "
        "With a deep understanding of the tech industry, you transform complex concepts into compelling narratives."
    ),
    verbose=True,
    allow_delegation=True,
    tools=[search_tool],
    cache=False,  # Disable cache for this agent
)

# Create tasks for your agents
task1 = Task(
    description=(
        "Conduct a comprehensive analysis of the latest AI research over the last week.. "
        "Identify key insights, breakthrough technologies, and potential industry impacts. "
        "Compile your findings in a detailed report. "
        "Make sure to check with a human if the draft is good before finalizing your answer."
    ),
    expected_output='A comprehensive report on the latest AI research over the last week, leave nothing out',
    agent=researcher,
    human_input=True
)

task2 = Task(
    description=(
        "Using the insights from the researcher\'s report, develop an engaging blog post that highlights the most significant AI advancements. "
        "Your post should be informative yet accessible, catering to a tech-savvy audience. "
        "Aim for a narrative that captures the essence of these breakthroughs and their implications for the future."
    ),
    expected_output='A compelling 3 paragraphs blog post formatted as markdown about the latest AI advancements',
    agent=writer
)

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=True,
    memory=True,
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

2024-08-10 15:31:48,023 - 7948459712 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-10 15:31:48][DEBUG]: == Working Agent: Senior Research Analyst
 [2024-08-10 15:31:48][INFO]: == Starting Task: Conduct a comprehensive analysis of the latest AI research over the last week.. Identify key insights, breakthrough technologies, and potential industry impacts. Compile your findings in a detailed report. Make sure to check with a human if the draft is good before finalizing your answer.


> Entering new CrewAgentExecutor chain...
To conduct a comprehensive analysis of the latest AI research over the last week, I will start by looking into the latest papers and research articles published in the AI domain. I will use tools to search the internet for the latest research papers, insights, and developments on AI from various sources. This will include:

1. Checking the latest submissions on Arxiv.
2. Exploring the daily papers on Hugging Face.
3. Conducting a broader search for any other significant AI research articles published recently.

Action: Search the internet


KeyboardInterrupt: 

In [3]:
# The search is not as targeted as I want it to be. It's not clear how the navigation happens here? 
# Quick combination with surfer might help here.